In [1]:
import pickle

In [2]:
import vk_api
vk_session = vk_api.VkApi(token="4ee251bf489b9d88106f08b36239eb0ab39bed07ca3fb9adacf62346cb2981eed59a26f4c29a18123f053")

In [3]:
vk_session.authorization()

In [4]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [27]:
def get_many_of_public(group_link, vk_session):
    global cached_ids
    
    shortname = group_link[group_link.rfind("/")+1:]
    r = vk_session.method("groups.getById", {"group_ids": shortname})
    group_id = r[0]["id"]
    
    if group_id not in cached_ids.keys():
    
        """Все сообщества подписчиков сообщества"""
        users = vk_api.VkTools(vk_session).get_all("groups.getMembers", 100, {'group_id':group_id})['items']
        k = int(len(users) / 1000)
        if k:
            users = users[::k]

        data_for_clustering = []
        users_publicpages = []
        with vk_api.VkRequestsPool(vk_session) as pool:
            
            for i in range(0, len(users), 1000):
                batch = users[i:i+1000]
                data_for_clustering.append(pool.method('users.get', {
                        "user_ids": batch,
                        "fields": "sex,age,education,universities,schools,interests,music,movies,bdate,city,country"
                    }))

                for user_id in batch:
                    users_publicpages.append(pool.method('users.getSubscriptions', {
                                "user_id": user_id
                            }))

        data_for_clustering = [x for x in [x.result for x in data_for_clustering]]
        t = []
        for user in users_publicpages:
            try:
                t.append(user.result['groups']['items'])
            except:
                pass
        users_publicpages = t
        
        wall50 = vk_session.method("wall.get", {"owner_id": -group_id, "filter": "owner", "count": 100})['items'][::2]
        
        result = data_for_clustering, users_publicpages, wall50
        cached_ids[group_id] = result
        pickle.dump(cached_ids, open("cache.p", "wb"))
        return 
    else:
        return cached_ids[group_id]

In [30]:
from time import time
t = time()
a = get_many_of_public("https://vk.com/goto_msk", vk_session)
print(time() - t)

Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
Got error while executing request
108.83647108078003


In [31]:
len(cached_ids[80270762][1])

972

In [3]:
import pickle

In [4]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [5]:
from sklearn.manifold import TSNE

In [6]:
tsne = TSNE()

In [7]:
cached_ids.keys()

dict_keys([77711747, 11047300, 395094, 22067335, 57396876, 22902287, 43037456, 53966739, 28481050, 42805402, 33171614, 62185521, 80270762, 51209390, 23709617, 38000341, 52809908, 63339192, 10710452, 43248954, 58809407, 23775044, 29875269, 31955143, 515529, 28469067, 28878800, 14947155, 48746452, 46547413, 44744406, 31068378, 27370717, 59012191, 7395425, 15323502, 42168573, 37056240, 23513843, 47038708, 38772728, 5364985, 7988605, 62731774, 55384959])

In [8]:
import pandas as pd
data = pd.read_csv("communities.csv").to_dict()

In [15]:
indexer = {v: k for k, v in data["link"].items()}
y = {}
for group_link in list(indexer.keys())[1:]:
    shortname = group_link[group_link.rfind("/")+1:]
    if "public" in shortname:
        shortname = shortname[6:]
    y[shortname] = indexer[group_link]
indexer = y

In [19]:
import vk
session = vk.Session()
api = vk.API(session)

x = {}
for k, v in indexer.items():
    try:
        r = api.groups.getById(group_ids=k)
        group_id = r[0]["gid"]
        print(group_id)
        x[group_id] = v
    except:
        pass
indexer = x

46337218
32588450
59793784
49684148
13818
38378240
51032338
61169188
45198828
49008081
36287833
5889089
13158464
46091804
53966394
18999205
16130329
217409
39576314
72948083
27612384
48984363
375595
40865377
42094829
11047300
189165
17226077
49752197
18130170
38289957
14176227
65472636
21222282
44766328
38243551
34479639
2005
25334190
69571501
25946580
5608057
41827660
40897846
247381
28349500
142116
76892879
10751825
31793007
30362362
64607113
15503435
65946753
25244932
28748235
64492852
17348990
31189947
54478267
29987531
48126202
37140977
27846618
77610102
20492787
23148107
51643003
43776215
2635139
81933666
48040432
34491914
15327516
56608785
84204021
50012518
38344204
19232159
101411767
31691361
11914597
42503348
6874027
59971648
56885475
47395
46813166
15147181
71848083
24123546
25165799
334424
88155848
22913517
610
52809908
84367613
58294532
25626906
41035449
28857658
12612178
68481433
13569130
64250751
32716331
17820732
18769596
39939573
37785226
6912839
36792820
6749510
294975

In [24]:
indexer = pickle.load(open("community_indexer.p", "rb"))

In [ ]:
tsne.fit_transform()